In [ ]:
proj_list =[
    'boringssl', 'c-ares',
    'freetype2', 'guetzli',
    'harfbuzz', 'libpng',
    'libssh', 'libxml2',
    'pcre', 'proj4',
    'r32', 'sqlite3',
    'vorbis', 'woff2',
    'wpantund'
]

version = 'version3'
proj_list = [
    'total_aspell', 'total_boringssl', 'total_c-ares', 'total_exiv2',
    'total_freetype2', 'total_grok', 'total_guetzli', 'total_harfbuzz',
    'total_lcms', 'total_libarchive', 'total_libexif', 'total_libhtp',
    'total_libpng', 'total_libsndfile', 'total_libssh', 'total_libxml2',
    'total_ndpi', 'total_openthread', 'total_pcre2', 'total_proj4',
    'total_re2', 'total_sqlite3', 'total_usrsctp', 'total_vorbis',
    'total_woff2', 'total_wpantund', 'total_yara', 'total_zstd'
]

In [ ]:
target_project = 0

In [ ]:
from sklearn.model_selection import train_test_split
import torch
import torch, gc

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import timeit

import data
import data_loader as dl
import initializer as init
import trainer
import tester
# import predictor
import model_util as mu

import os

In [ ]:
gc.collect()
torch.cuda.empty_cache()

# print(torch.cuda.memory_summary(device=None, abbreviated=False))

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
'''
{
    "prefix":[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 110, 102, 123, 54],
    
    "prefix-ids":["", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "TBBox_Rec_", ""],
    
    "postfix": [123, 37, 123, 72, 123, 8, 123, 85, 123, 72, 123, 8, 123, 48, 123, 8, 123, 55, 52, 91, 123, 8, 123, 37, 123, 72, 123, 8, 123, 85, 123, 72, 123, 8, 123, 57, 123, 8, 123, 55, 52, 54, 30, 54, 85, 123, 97, 123, 22, 123, 97, 123, 55, 123, 53, 99, 91, 123, 84, 91, 123, 123, 91, 123],
    
    "postfix-ids": ["to", "", "xMax", "", "bbox", "", "user", "", "xMax", "", "bbox", "", "user", "", "x", "", "to", "", "", "", "x", "", "to", "", "xMin", "", "bbox", "", "user", "", "xMin", "", "bbox", "", "user", "", "x", "", "to", "", "", "", "", "", "", "user", "", "TBBox_Rec", "", "to", "", "FT_Vector", "", "BBox_Move_To", "", "", "", "TBBox_Rec", "", "", "bbox", "FT_BBox", "", "last"],
    
    "label-type": [123],
    
    "label-prefix": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
    
    "label-postfix": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
    
    "case": 2
}
'''

In [ ]:
# get all data exept target project
prefix_np, prefix_ids_np,\
postfix_np, postfix_ids_np,\
label_np,\
label_prefix_np, label_postfix_np,\
case_np = data.getTrainData(proj_list, proj_list[target_project], version)

In [ ]:
# get target project data
'''
test_prefix, test_postfix,\
test_label, test_label_prefix,\
test_label_postfix, test_case = data.getTestData(proj_list[target_project])
'''

In [ ]:
train_prefix, test_prefix,\
train_prefix_ids, test_prefix_ids,\
train_postfix, test_postfix,\
train_postfix_ids, test_postfix_ids,\
train_label, test_label,\
train_label_prefix, test_label_prefix,\
train_label_postfix, test_label_postfix,\
train_case, test_case = train_test_split(
    prefix_np,
    prefix_ids_np,
    postfix_np,
    postfix_ids_np,
    label_np,
    label_prefix_np,
    label_postfix_np,
    case_np,
    test_size = 0.2, random_state = 43
)

In [ ]:
train_prefix, val_prefix,\
train_prefix_ids, val_prefix_ids,\
train_postfix, val_postfix,\
train_postfix_ids, val_postfix_ids,\
train_label, val_label,\
train_label_prefix, val_label_prefix,\
train_label_postfix, val_label_postfix,\
train_case, val_case = train_test_split(
    train_prefix,
    train_prefix_ids,
    train_postfix,
    train_postfix_ids,
    train_label,
    train_label_prefix,
    train_label_postfix,
    train_case,
    test_size = 0.2, random_state = 43
)

In [ ]:
print('train: ', len(train_label))
print('validation: ', len(val_label))
print('test: ', len(test_label))

In [ ]:
train_dataloader, val_dataloader, test_dataloader =\
    dl.data_loader(
        train_prefix, val_prefix, test_prefix,
        train_prefix_ids, val_prefix_ids, test_prefix_ids,

        train_postfix, val_postfix, test_postfix,
        train_postfix_ids, val_postfix_ids, test_postfix_ids,

        train_label, val_label, test_label,

        train_label_prefix, val_label_prefix, test_label_prefix,
        train_label_postfix, val_label_postfix, test_label_postfix,

        train_case, val_case, test_case,

        batch_size=1000
    )

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
# ====================
# set parameters here
# ====================

overall_title = 'version3_'
title = overall_title + '01'
epochs = 20 

# max_len, source_code_tokens, token_choices = data.getInfo()

learning_rate = 0.001
weight_decay = 0.0

embed_dim = 100 # 100
hidden_size = 200 # 200
n_layers = 1
output_size = 1 # max(token_choices) + 1
dropout = 0.0
max_length = 64 # max_len
input_size = 154 # max(token_choices) + 1
device = device

model_name = "seq2seq"
optim_name = "Adam"
loss_fn_name = "BCE"

teacher_forcing_ratio = 0.75
threshold = torch.tensor([0.5]).to(device)

In [ ]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/'+title+'/tests')

In [ ]:
trainer.set_seed(42)

model, loss_fn, optimizer = init.initialize_model(
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    n_layers=n_layers,
    output_size=output_size,
    dropout=dropout,
    max_length=max_length,
    input_size=input_size,
    device=device,
    loss_fn_name=loss_fn_name
)

print(model)

In [ ]:
start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    teacher_forcing_ratio=teacher_forcing_ratio,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    device=device,
    threshold=threshold
)

end_time = (timeit.default_timer() - start_time) / 60.0

In [ ]:
mu.saveModel(overall_title, title, model)

In [ ]:
model = mu.getModel(overall_title, title)
print(model)

In [ ]:
loss, acc, TT_acc = tester.test(
    test_dataloader=test_dataloader,
    model=model,
    loss_fn=loss_fn,
    device=device,
    fn=overall_title,
    proj_nm=title,
    threshold=threshold
)

In [ ]:
with open('../stat/'+overall_title, 'a') as f:
        text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\t |\t TT acc: ' + str(TT_acc)
        # text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(0.0, 3)) + ' min\t |\t TT acc: ' + str(TT_acc)
        f.write(text)

In [ ]:
mu.graphModel(train_dataloader, model, writer, device)